---

Load libraries

---

In [1]:
## Load libraries
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from sklearn.metrics import confusion_matrix
plt.style.use('dark_background')
%matplotlib inline

---

Set printing precision

---

In [2]:
np.set_printoptions(precision = 2)

---

Mount Google drive

---

In [ ]:
## Mount Google drive folder if running in Colab
if('google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    DIR = '/content/drive/MyDrive/Colab Notebooks/EvenSemester2024/DL/Codes'
    DATA_DIR = DIR + '/Data/'
else:
    DATA_DIR = 'Data/'

Mounted at /content/drive


---

Load MNIST Data

---

In [ ]:
## Load MNIST data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])

num_labels = len(np.unique(y_train))
num_features = X_train.shape[1]
num_samples = X_train.shape[0]

# One-hot encode class labels
Y_train = tf.keras.utils.to_categorical(y_train)
Y_test = tf.keras.utils.to_categorical(y_test)

# Normalize the samples (images) using the training data
xmax = np.amax(X_train) # 255
xmin = np.amin(X_train) # 0
X_train = (X_train - xmin) / (xmax - xmin) # all train features turn into a number between 0 and 1
X_test = (X_test - xmin) / (xmax - xmin)

print('MNIST set')
print('---------------------')
print('Number of training samples = %d'%(num_samples))
print('Number of features = %d'%(num_features))
print('Number of output labels = %d'%(num_labels))

---

We consider a softmax classfier, which is a 1-layer neural network or a 0-hidden layer neural network, for a batch comprising $b$ samples represented as the $b\times 784$-matrix $$\mathbf{X} = \begin{bmatrix}{\mathbf{x}^{(0)}}^\mathrm{T}\\{\mathbf{x}^{(1)}}^\mathrm{T}\\\vdots\\{\mathbf{x}^{(b-1)}}^\mathrm{T}\end{bmatrix}$$ with one-hot encoded true labels represented as the $b\times 10$-matrix (10 possible categories) $$\mathbf{Y}=\begin{bmatrix}{\mathbf{y}^{(0)}}^\mathrm{T}\\{\mathbf{y}^{(1)}}^\mathrm{T}\\\vdots\\{\mathbf{y}^{(b-1)}}^\mathrm{T}\end{bmatrix}.$$

The forward propagation for a generic sample in the batch seen as a $1\times784$-object $\mathbf{x}^\mathrm{T}$ with the bias feature $1$ added is presented below:

$$\small\begin{align*}
\boxed{\underbrace{\mathbf{x}_B^\mathrm{T}}_{1\times785}=\begin{bmatrix}\mathbf{x}^\mathrm{T}&1\end{bmatrix}}&\rightarrow\boxed{\underbrace{{\mathbf{z}}^\mathrm{T}}_{1\times 10} = \underbrace{\mathbf{x}_B^\mathrm{T}}_{1\times785}\underbrace{{\mathbf{W}}}_{785\times10}}\rightarrow\boxed{\underbrace{{\mathbf{a}}^\mathrm{T}}_{1\times10}=\text{softmax}\left(\underbrace{{\mathbf{z}}^\mathrm{T}}_{1\times10}\right)}\rightarrow\boxed{L = \sum\limits_{k=0}^9-y_k\log\left(\hat{y}_k\right)}.
\end{align*}$$

The forward propagation for the same generic sample seen as a $784$-vector $\mathbf{x}$ with the bias feature $1$ added is presented below (note that the weight matrix has the same name $\mathbf{W}$ as above for simplicity even though it should show up as $\mathbf{W}^\mathrm{T}$):

$$\small\begin{align*}
\boxed{\underbrace{\mathbf{x}_B}_{785}=\begin{bmatrix}\mathbf{x}\\1\end{bmatrix}}&\rightarrow\boxed{\underbrace{\mathbf{z}}_{10} = \underbrace{\mathbf{W}}_{10\times785}\underbrace{\mathbf{x}_B}_{785}}\rightarrow\boxed{\underbrace{\mathbf{a}}_{10}=\text{softmax}\left(\underbrace{\mathbf{z}}_{10}\right)}\rightarrow\boxed{L = \sum\limits_{k=0}^9-y_k\log\left(\hat{y}_k\right)}.\end{align*}$$

We will derive the update rule for the weights matrix $\mathbf{W}$ using the setup above.


The average crossentropy (CCE) loss for the batch is:$$\begin{align*}L &=\frac{1}{b}\left[L_0+L_1+\cdots+L_{b-1}\right]\\&=\frac{1}{b}\left[\sum\limits_{k=0}^9{\color{yellow}-}y_k^{(0)}\log\left(\hat{y}^{(0)}_k\right)+\sum\limits_{k=0}^9{\color{yellow}-}y_k^{(1)}\log\left(\hat{y}^{(1)}_k\right)+\cdots+\sum\limits_{k=0}^9{\color{yellow}-}y_k^{(b-1)}\log\left(\hat{y}^{(b-1)}_k\right)\right]\\&=\frac{1}{b}\left[{\color{yellow}-}{\mathbf{y}^{(0)}}^\mathrm{T}\log\left({\hat{\mathbf{y}}^{(0)}}\right)+{\color{yellow}-}{\mathbf{y}^{(1)}}^\mathrm{T}\log\left({\hat{\mathbf{y}}^{(1)}}\right)+\cdots+{\color{yellow}-}{\mathbf{y}^{(b-1)}}^\mathrm{T}\log\left({\hat{\mathbf{y}}^{(b-1)}}\right)\right].\end{align*}$$

The computational graph for the samples, each at a time treated as a $785$-vector, in the batch are presented below where the weights matrix has shape $10\times 785.$

$\hspace{1.5in}\begin{align*}L_0\\{\color{yellow}\downarrow}\\ \hat{\mathbf{y}}^{(0)} &= \mathbf{a}^{(0)}\\{\color{yellow}\downarrow}\\\mathbf{z}^{(0)}\\{\color{yellow}\downarrow}\\\mathbf{W}\end{align*}$$\hspace{0.25in}\begin{align*}L_1\\{\color{yellow}\downarrow}\\ \hat{\mathbf{y}}^{(1)} &= \mathbf{a}^{(1)}\\{\color{yellow}\downarrow}\\\mathbf{z}^{(1)}\\{\color{yellow}\downarrow}\\\mathbf{W}\end{align*}$$\qquad\cdots\qquad$$\begin{align*} L_{b-1}\\{\color{yellow}\downarrow}\\ \hat{\mathbf{y}}^{(b-1)} &= \mathbf{a}^{(b-1)}\\{\color{yellow}\downarrow}\\\mathbf{z}^{(b-1)}\\{\color{yellow}\downarrow}\\\mathbf{W}\end{align*}$

The gradient of the average batch loss w.r.t. the weights is:
$$\small\begin{align*}\Rightarrow \nabla_\mathbf{W}(L) &=\frac{1}{b}\left[\nabla_\mathbf{W}(L_0)+\nabla_\mathbf{W}(L_1)+\cdots+\nabla_\mathbf{W}(L_{b-1})\right]\end{align*}$$
which by chain rule can be written as:

$$\small\begin{align*}\Rightarrow \nabla_\mathbf{W}(L) &= \frac{1}{b}\left(\underbrace{\left[\nabla_\mathbf{W}\left(\mathbf{z}^{(0)}\right) \times\nabla_{\mathbf{z}^{(0)}}\left(\hat{\mathbf{y}}^{(0)}\right)\times\nabla_{\hat{\mathbf{y}}^{(0)}}(L_0)\right]}_{\nabla_\mathbf{W}(L_0)}+\underbrace{\left[\nabla_\mathbf{W}\left(\mathbf{z}^{(1)}\right) \times\nabla_{\mathbf{z}^{(1)}}\left(\hat{\mathbf{y}}^{(1)}\right)\times\nabla_{\hat{\mathbf{y}}^{(1)}}(L_1)\right]}_{\nabla_\mathbf{W}(L_1)}+\cdots+\underbrace{\left[\nabla_\mathbf{W}\left(\mathbf{z}^{(b-1)}\right) \times\nabla_{\mathbf{z}^{(b-1)}}\left(\hat{\mathbf{y}}^{(b-1)}\right)\times\nabla_{\hat{\mathbf{y}}^{(b-1)}}(L_{b-1})\right]}_{\nabla_\mathbf{W}(L_{b-1})}\right)\\&=\frac{1}{b}\left(\underbrace{\left[\nabla_\mathbf{W}\left(\mathbf{z}^{(0)}\right) \times\nabla_{\mathbf{z}^{(0)}}\left({\mathbf{a}}^{(0)}\right)\times\nabla_{\hat{\mathbf{y}}^{(0)}}(L_0)\right]}_{\nabla_\mathbf{W}(L_0)}+\underbrace{\left[\nabla_\mathbf{W}\left(\mathbf{z}^{(1)}\right) \times\nabla_{\mathbf{z}^{(1)}}\left({\mathbf{a}}^{(1)}\right)\times\nabla_{\hat{\mathbf{y}}^{(1)}}(L_1)\right]}_{\nabla_\mathbf{W}(L_1)}+\cdots+\underbrace{\left[\nabla_\mathbf{W}\left(\mathbf{z}^{(b-1)}\right) \times\nabla_{\mathbf{z}^{(b-1)}}\left(\hat{\mathbf{y}}^{(b-1)}\right)\times\nabla_{\hat{\mathbf{y}}^{(b-1)}}(L_{b-1})\right]}_{\nabla_\mathbf{W}(L_{b-1})}\right)\\&=\frac{1}{b}\sum_{i=0}^{b-1}\left[\nabla_\mathbf{W}\left(\mathbf{z}^{(i)}\right) \times\nabla_{\mathbf{z}^{(i)}}\left({\mathbf{a}}^{(i)}\right)\times\nabla_{\hat{\mathbf{y}}^{(i)}}(L_i)\right]\\&=\frac{1}{b}\sum_{i=0}^{b-1}\left[\nabla_\mathbf{W}\left(\mathbf{W}{\mathbf{x}^{(i)}_B}\right) \times\nabla_{\mathbf{z}^{(i)}}\left(\text{softmax}\left({\mathbf{z}}^{(i)}\right)\right)\times\nabla_{\hat{\mathbf{y}}^{(i)}}\left(-{\mathbf{y}^{(i)}}^\mathrm{T}\log\left(\hat{\mathbf{y}}^{(i)}\right)\right)\right],\end{align*}$$
which can be written as

$$\begin{align*}\nabla_{\mathbf{W}}(L) &= \dfrac{1}{b}\displaystyle\sum_{i=0}^{b-1}\underbrace{\begin{bmatrix}\boxed{{\mathbf{x}^{(i)}_B}\ \pmb{0}\ \pmb{0}\ \ldots\ \pmb{0}}&&&&\\\\&\boxed{\pmb{0}\ {\mathbf{x}^{(i)}_B}\ \pmb{0}\ \ldots\ \pmb{0}}&&&\\&\hspace{1cm}\ddots&&&\\&&\hspace{-0.5cm}\ddots&&\\&&&\boxed{\pmb{0}\ \pmb{0}\ \ldots\ \pmb{0}\ {\mathbf{x}^{(i)}_B}}&\end{bmatrix}}_{\color{cyan}{\nabla_\mathbf{W}\left(\mathbf{z}^{(i)}\right)=\nabla_\mathbf{W}\left(\mathbf{W}{\mathbf{x}^{(i)}_B}\right):\ 10\times785\times10}}\underbrace{\begin{bmatrix}a^{(i)}_0 (1 - a^{(i)}_0) & -a^{(i)}_0 a^{(i)}_1 & \cdots & -a^{(i)}_0 a^{(i)}_9\\-a^{(i)}_1 a^{(i)}_0 & a^{(i)}_1 (1 - a^{(i)}_1) & \cdots & -a^{(i)}_1 a^{(i)}_9\\\vdots & \vdots & \ddots & \vdots\\-a^{(i)}_9 a^{(i)}_0 & -a^{(i)}_9 a^{(i)}_1 & \cdots & a^{(i)}_9 (1 - a^{(i)}_9)\end{bmatrix}}_{\color{cyan}{\nabla_{\mathbf{z}^{(i)}}\left({\mathbf{a}}^{(i)}\right) = \nabla_{\mathbf{z}^{(i)}}\left(\text{softmax}\left({\mathbf{z}}^{(i)}\right)\right):\ 10\times10}}\underbrace{\begin{bmatrix}-\frac{y^{(i)}_0}{\hat{y}^{(i)}_0}\\-\frac{y^{(i)}_1}{\hat{y}^{(i)}_1}\\\vdots\\-\frac{y^{(i)}_9}{\hat{y}^{(i)}_9}\end{bmatrix}}_{\color{cyan}{\nabla_{\hat{\mathbf{y}}^{(i)}}(L_i)=\nabla_{\hat{\mathbf{y}}^{(i)}}\left(-{\mathbf{y}^{(i)}}^\mathrm{T}\log\left(\hat{\mathbf{y}}^{(i)}\right)\right):\ 10\times1}}\end{align*}$$

The forward and backward propagation showing the gradient flow for a generic sample is shown below:

![](https://1drv.ms/i/c/37720f927b6ddc34/IQS3b-biQ4W9QpCtJzaZnyCoAQ8_r9i707rpOE1O9I0yntM?width=686&height=93)

$$\begin{align*}\nabla_{\mathbf{W}}(L) &=\dfrac{1}{b}\displaystyle\sum_{i=0}^{b-1}\underbrace{\begin{bmatrix}a^{(i)}_0 (1 - a^{(i)}_0) & -a^{(i)}_0 a^{(i)}_1 & \cdots & -a^{(i)}_0 a^{(i)}_9\\-a^{(i)}_1 a^{(i)}_0 & a^{(i)}_1 (1 - a^{(i)}_1) & \cdots & -a^{(i)}_1 a^{(i)}_9\\\vdots & \vdots & \ddots & \vdots\\-a^{(i)}_9 a^{(i)}_0 & -a^{(i)}_9 a^{(i)}_1 & \cdots & a^{(i)}_9 (1 - a^{(i)}_9)\end{bmatrix}}_{\color{cyan}{=\left(\mathbf{I}-{\mathbf{a}^{(i)}}^\mathrm{T}\right)\otimes\mathbf{a}^{(i)}}}\underbrace{\begin{bmatrix}-\frac{y^{(i)}_0}{\hat{y}^{(i)}_0} \\
-\frac{y^{(i)}_1}{\hat{y}^{(i)}_1}\\\vdots\\-\frac{y^{(i)}_9}{\hat{y}^{(i)}_9}\end{bmatrix}}_{\color{cyan}{=-\frac{\mathbf{y}^{(i)}}{\hat{\mathbf{y}}^{(i)}}}}{\mathbf{x}^{(i)}_B}^\mathrm{T}.\end{align*}$$

We can write the gradient in the following way for efficient coding purposes: $$\nabla_\mathbf{W}(L) = \frac{1}{b}\sum_{i=0}^{b-1}\left[\left(\mathbf{I}-{\mathbf{a}^{(i)}}^\mathrm{T}\right)\otimes\mathbf{a}^{(i)}\right]\left[-\frac{\mathbf{y}^{(i)}}{\hat{\mathbf{y}}^{(i)}}\right]{\mathbf{x}^{(i)}_B}^\mathrm{T}.$$



It can be seen that the gradient object has shape $(10\times 10)\times(10\times1)\times(1\times785)=10\times785,$ which is the same shape as the weights matrix $\mathbf{W}.$ However, our derivation here assumed that the samples are seen as column vectors of the data matrix. The original data matrix has the samples arranged as rows which corresponded to the weights matrix of shape $785\times10.$ In order to get the gradient w.r.t. that weights matrix, we have to transpose this expression resulting in the update $$\nabla_\mathbf{W}(L) = \frac{1}{b}\sum_{i=0}^{b-1}\underbrace{\mathbf{x}^{(i)}_B}_{\color{yellow}{785\times1}}\underbrace{\underbrace{\left[-\frac{{\mathbf{y}^{(i)}}^\mathrm{T}}{{\hat{\mathbf{y}}^{(i)}}^\mathrm{T}}\right]}_{\color{magenta}{\text{output side gradient of softmax layer: }1\times10}}\underbrace{\left[\left(\mathbf{I}-{\mathbf{a}^{(i)}}\right)\otimes{\mathbf{a}^{(i)}}^\mathrm{T}\right]}_{\color{magenta}{\text{local gradient of softmax layer: }10\times10}}}_{\color{yellow}{\text{output side gradient of dense layer: }1\times10}}.$$




---

---

A generic layer class with forward and backward methods

----

In [ ]:
class Layer:
  def __init__(self):
    self.input = None
    self.output = None

  def forward(self, input):
    pass

  def backward(self, output_gradient, learning_rate):
    pass

---

CCE loss and its gradient for the batch samples

---

In [ ]:
## Define the loss function and its gradient
def CCE(Y, Yhat):
  return(np.mean(np.sum(-Y*np.log(Yhat), axis = 1)))
  # TensorFlow in-built functions
  #cce = tf.keras.losses.CategoricalCrossentropy()
  #return(cce(Y, Yhat).numpy())

def cce_gradient(Y, Yhat):
  return(-Y/Yhat)

---

Softmax activation layer class


---

In [ ]:
## Softmax activation layer class
class Softmax(Layer):
  def forward(self, input):
    self.input = input
    self.output = tf.nn.softmax(self.input, axis = 1).numpy()

  def backward(self, output_gradient, learning_rate= None):
    I = np.identity(self.output.shape[1])
    local_gradient = (I - self.output[:, :, np.newaxis]) * self.output[:, np.newaxis, :]
    return(np.einsum('ij, ijk -> ik', output_gradient, local_gradient))


---

Dense layer class

---

In [ ]:
## Dense layer class
class Dense(Layer):
  def __init__(self, input_size, output_size):
    self.weights = 0.01*np.random.randn(input_size+1, output_size) # Bias trick
    self.weights[-1, :] = 0.01 # set all the bias values to the same nonzero constant

  def forward(self, input):
    self.input = np.hstack([input, np.ones((input.shape[0], 1))]) # bias feature as last column
    self.output = np.dot(self.input, self.weights)

  def backward(self, output_gradient, learning_rate):
    # Calculate gradient w.r.t dense layer weights from all input samples in the batch
    weights_gradient = (1/output_gradient.shape[0]) * np.einsum('ij, ik -> jk', self.input, output_gradient)
    # Update weights for dense layer
    self.weights = self.weights + learning_rate*(-weights_gradient)



---

Function to generate sample indices for batch processing according to batch size

---

In [ ]:
## Function to generate sample indices for batch processing according to batch size
def generate_batch_indices(num_samples, batch_size):
  # Reorder sample indices
  reordered_sample_indices = np.random.choice(num_samples, num_samples, replace = False)
  # Generate batch indices for batch processing
  batch_indices = np.split(reordered_sample_indices, np.arange(batch_size, len(reordered_sample_indices), batch_size))
  return(batch_indices)

---

Example generation of batch indices

---

In [ ]:
## Example generation of batch indices
batch_size = 4
batch_indices = generate_batch_indices(32, batch_size)
print(batch_indices)

[array([12, 15, 10, 31]), array([11, 13, 30, 24]), array([ 6,  4, 28, 22]), array([ 9,  3, 18, 21]), array([ 5, 27,  8, 23]), array([19, 17,  0, 26]), array([29, 16,  7,  1]), array([ 2, 20, 25, 14])]


---

Train the 1-layer neural (softmax) neural network using batch training with batch size = 100

---

In [ ]:
## Train the 1-layer neural network using batch training with batch size = 100
learning_rate = ?
batch_size = ?
nepochs = ?
# Create empty array to store training losses over each epoch
loss_train_epoch = np.empty(nepochs, dtype = np.float64)
# Create empty array to store test losses over each epoch
loss_test_epoch = np.empty(nepochs, dtype = np.float64)

# Neural network architecture ()
dlayer1 = ? # define dense layer 1
softmax = ? # define softmax activation layer

# Steps: run over each sample in the batch, calculate loss, gradient of loss,
# and update weights.
epoch = 0
# Run over each epoch
while epoch < nepochs:
  # Generate the batches
  batch_indices = generate_batch_indices(num_samples, batch_size)
  loss = 0
  # Run over each batch of samples
  for b in range(len(?)):
    # Forward prop starts here
    ?
    # Forward prop ends and backward prop starts here
    ?
  # Calculate the average training loss for the current epoch
  loss_train_epoch[epoch] = ?

  # Forward propagation for test data
  ?

  # Calculate test data loss
  loss_test_epoch[epoch] = ?

  print('Epoch %d: train loss = %f, test loss = %f'%(epoch+1, loss_train_epoch[epoch], loss_test_epoch[epoch]))
  epoch = epoch + 1

---

Plot training and test loss vs. epoch

---

In [ ]:
## Plot train and test loss as a function of epoch:
fig, ax = plt.subplots(1, 1, figsize = (4, 4))
fig.tight_layout(pad = 4.0)
ax.plot(loss_train_epoch, 'b', label = 'Train')
ax.plot(loss_test_epoch, 'r', label = 'Test')
ax.set_xlabel('Epoch', fontsize = 12)
ax.set_ylabel('Loss value', fontsize = 12)
ax.legend()
ax.set_title('Loss vs. Epoch for Softmax Classifier', fontsize = 14);

---

Assess model performance on test data

---

In [ ]:
## Assess model performance on test data
?
ypred = ?
ytrue = ?
print('Accuracy on test data = %3.2f'%(np.mean(ytrue == ypred)*100))
# Print confusion matrix
print(confusion_matrix(ytrue, ypred))

In [ ]:
## Plot a random test sample with its predicted label printed above the plot
test_index = np.random.choice(X_test.shape[0])
fig, ax = plt.subplots(1, 1, figsize = (2, 2))
print(f'Image classified as {ypred[test_index]}')
ax.imshow(tf.reshape(X_test[test_index], [28, 28]).numpy(), cmap = 'gray');